In [19]:
# transfer learning with tensorflow : Scaling Up

# **IMPORTING HELPER FUNCTION**

In [20]:
 !wget "https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py"

--2024-08-22 04:55:20--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2024-08-22 04:55:20 (86.2 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [21]:
#import series of helper functions for the notebook (we've created/used these in previous notebooks)
from helper_functions import create_tensorboard_callback, plot_loss_curves, unzip_data, compare_historys, walk_through_dir

## 101 food classes : working with less data(10%)
our goal is to beat the orginal food101 with 10% of the training data.
downloading that data

In [28]:
!wget "https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip"
unzip_data("101_food_classes_10_percent.zip")

--2024-08-22 05:00:00--  https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.26.207, 173.194.212.207, 172.217.204.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.26.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1625420029 (1.5G) [application/zip]
Saving to: ‘101_food_classes_10_percent.zip.1’

101_food_classes_10 100%[===================>]   1.51G   121MB/s    in 21s     

2024-08-22 05:00:20 (75.2 MB/s) - ‘101_food_classes_10_percent.zip.1’ saved [1625420029/1625420029]



In [29]:
train_dir="101_food_classes_10_percent/train/"
test_dir="101_food_classes_10_percent/test/"

In [37]:
walk_through_dir("101_food_classes_10_percent/")

There are 3 directories and 0 images in '101_food_classes_10_percent/'.
There are 0 directories and 0 images in '101_food_classes_10_percent/.ipynb_checkpoints'.
There are 102 directories and 0 images in '101_food_classes_10_percent/test'.
There are 0 directories and 250 images in '101_food_classes_10_percent/test/tacos'.
There are 0 directories and 250 images in '101_food_classes_10_percent/test/red_velvet_cake'.
There are 0 directories and 250 images in '101_food_classes_10_percent/test/breakfast_burrito'.
There are 0 directories and 250 images in '101_food_classes_10_percent/test/apple_pie'.
There are 0 directories and 250 images in '101_food_classes_10_percent/test/lobster_roll_sandwich'.
There are 0 directories and 250 images in '101_food_classes_10_percent/test/pork_chop'.
There are 0 directories and 250 images in '101_food_classes_10_percent/test/panna_cotta'.
There are 0 directories and 250 images in '101_food_classes_10_percent/test/spaghetti_bolognese'.
There are 0 directorie

In [31]:
# Setup data inputs
import tensorflow as tf
IMG_SIZE = (224, 224)
train_data_all_10_percent = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                                label_mode="categorical",
                                                                                image_size=IMG_SIZE)

test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                label_mode="categorical",
                                                                image_size=IMG_SIZE,
                                                                shuffle=False) # don't shuffle test data for prediction analysis


Found 7575 files belonging to 101 classes.
Found 32825 files belonging to 102 classes.


In [32]:
# setup input data
import tensorflow as tf
IMG_SIZE=(224,224)
train_data_all_10_percent=tf.keras.preprocessing.image_dataset_from_directory(directory=train_dir,
                                                                              image_size=IMG_SIZE,
                                                                              label_mode="categorical")
test_data_all_10_percent=tf.keras.preprocessing.image_dataset_from_directory(directory=test_dir,
                                                                              image_size=IMG_SIZE,
                                                                              label_mode="categorical",
                                                                             shuffle=False)

Found 7575 files belonging to 101 classes.
Found 32825 files belonging to 102 classes.


# **Train a big dog model with transfer learning on 10% of 101 food classes**

**STEPS:**

1. A ModelCheckpoint callback to save our progress during training, this means we could experiment with further training later without having to train from scratch every time.

2. Data augmentation built right into the model
3. A headless (no top layers) EfficientNetB0 architecture from tf.keras.applications as our base model

4. A Dense layer with 101 hidden neurons (same as number of food classes) and softmax activation as the output layer
5. Categorical crossentropy as the loss function since we're dealing with more than two classes
6. The Adam optimizer with the default settings
7. Fitting for 5 full passes on the training data while evaluating on 15% of the test data




In [40]:
# Step1:  Create checkpoint callback to save model for later use
checkpoint_path = "101_classes_10_percent_data_model_checkpoint.weights.h5"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                         save_weights_only=True, # save only the model weights
                                                         monitor="val_accuracy", # save the model weights which score the best validation accuracy
                                                         save_best_only=True) # only keep the best model weights on file (delete the rest)


In [46]:
# Step2: Data Augmentation
from tensorflow.keras import layers
from tensorflow.keras import Sequential

# setup data augmentation
data_augmentation=Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.2),
    layers.RandomHeight(0.2),
    layers.RandomZoom(0.2),
    layers.RandomWidth(0.2)
],name="data_augmentation")

In [51]:
# Step 3: A headless (no top layers) EfficientNetB0 architecture from tf.keras.applications as our base model

# setting our base model
base_model=tf.keras.applications.efficientnet.EfficientNetB0(include_top=False)
base_model.trainable=False

# setting model architecture with top trainable layer
input=layers.Input(shape=(224,224,3),name="input_layer")
x=data_augmentation(input)
x=base_model(x,training=False)
x=layers.GlobalAveragePooling2D(name="global_avg_pool_layer")(x)
output=layers.Dense(len(train_data_all_10_percent.class_names),activation="softmax",name="output_layer")(x)

model=tf.keras.Model(input,output)

In [52]:
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ data_augmentation (Sequential)       │ (None, None, None, 3)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ efficientnetb0 (Functional)          │ (None, None, None, 1280)    │       4,049,571 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_avg_pool_layer                │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ output_layer (Dense)                 │ (None, 101)                 │         129,381 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,178,952 (15.94 MB)

 Trainable params: 129,381 (505.39 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [60]:
# compiling our model
model.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])
batch_size=32
# fitting the model
steps_per_epoch = int(len(train_data_all_10_percent) // batch_size)
validation_steps = int(len(test_data) // batch_size)

history_all_classes_10_percent = model.fit( train_data_all_10_percent,
                                            epochs=5,
                                            validation_data=test_data,
                                            steps_per_epoch=steps_per_epoch,
                                            validation_steps=validation_steps)


Epoch 1/5
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.4398 - loss: 2.5807

ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 102), output.shape=(None, 101)